In [5]:
"""
The general LDA approach is very similar to a Principal Component Analysis  
but in addition to finding the component axes that maximize the variance of our data (PCA), 
we are additionally interested in the axes that maximize the separation between multiple classes (LDA).
Both Linear Discriminant Analysis (LDA) and Principal Component Analysis (PCA) are linear transformation techniques 
that are commonly used for dimensionality reduction. PCA can be described as an “unsupervised” algorithm, 
since it “ignores” class labels and its goal is to find the directions (the so-called principal components) 
that maximize the variance in a dataset. In contrast to PCA, LDA is “supervised” and computes the 
directions (“linear discriminants”) that will represent the axes that that maximize the separation between multiple classes.

Although it might sound intuitive that LDA is superior to PCA for a multi-class classification task where the 
class labels are known, this might not always the case.
For example, comparisons between classification accuracies for image recognition after using PCA or LDA show that PCA 
tends to outperform LDA if the number of samples per class is relatively small. In practice, it is also not uncommon
to use both LDA and PCA in combination: E.g., PCA for dimensionality reduction followed by an LDA.
"""

import pandas as pd
 
df = pd.io.parsers.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', 
    header=None, 
    sep=',', 
    )
feature_dict = {i:label for i,label in zip(
            range(4),
              ('sepal length in cm', 
              'sepal width in cm', 
              'petal length in cm', 
              'petal width in cm', ))}
df.columns = [l for i,l in sorted(feature_dict.items())] + ['class label']
df.dropna(how="all", inplace=True) # to drop the empty line at file-end
 
df.tail()

,sepal length in cm,sepal width in cm,petal length in cm,petal width in cm,class label
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [6]:
from sklearn.preprocessing import LabelEncoder
 
X = df[[0,1,2,3]].values 
y = df['class label'].values
 
enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1
 
label_dict = {1: 'Setosa', 2: 'Versicolor', 3:'Virginica'}

In [7]:
from matplotlib import pyplot as plt
import numpy as np
import math
 
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,6))
 
for ax,cnt in zip(axes.ravel(), range(4)):
 
    # set bin sizes
    min_b = math.floor(np.min(X[:,cnt]))
    max_b = math.ceil(np.max(X[:,cnt]))
    bins = np.linspace(min_b, max_b, 25)
 
    # plottling the histograms
    for lab,col in zip(range(1,4), ('blue', 'red', 'green')):
        ax.hist(X[y==lab, cnt],
                   color=col, 
                   label='class %s' %label_dict[lab], 
                   bins=bins,
                   alpha=0.5,)
    ylims = ax.get_ylim()
 
    # plot annotation
    leg = ax.legend(loc='upper right', fancybox=True, fontsize=8)
    leg.get_frame().set_alpha(0.5)
    ax.set_ylim([0, max(ylims)+2])
    ax.set_xlabel(feature_dict[cnt])
    ax.set_title('Iris histogram #%s' %str(cnt+1))
 
    # hide axis ticks
    ax.tick_params(axis="both", which="both", bottom="off", top="off",  
            labelbottom="on", left="off", right="off", labelleft="on")
 
    # remove axis spines
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False) 
    ax.spines["bottom"].set_visible(False) 
    ax.spines["left"].set_visible(False)
 
axes[0][0].set_ylabel('count')
axes[1][0].set_ylabel('count')
 
fig.tight_layout()
 
plt.show()

In [9]:
from sklearn import preprocessing
preprocessing.scale(X, axis=0, with_mean=True, with_std=True, copy=False)
#step 1 : compute the d dimension mean vector
np.set_printoptions(precision=4)
 
mean_vectors = []
for cl in range(1,4):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
    print('Mean Vector class %s: %s\n' %(cl, mean_vectors[cl-1]))
    

# Compute the scatter matrices

# 1 Sw within class scatter matrix
S_W = np.zeros((4,4))
for cl,mv in zip(range(1,4), mean_vectors):
    class_sc_mat = np.zeros((4,4))                  # scatter matrix for every class
    for row in X[y == cl]:
        row, mv = row.reshape(4,1), mv.reshape(4,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat                             # sum class scatter matrices
print('within-class Scatter Matrix:\n', S_W)


# 2 Between class matrix
overall_mean = np.mean(mean_vectors, axis=0)
 
S_B = np.zeros((4,4))
for i,mean_vec in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(4,1) # make column vector
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)
 
print('between-class Scatter Matrix:\n', S_B)


#solving the generalized eignevalue problem for the matrix Sw(inverse)Sb
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
 
for i in range(len(eig_vals)):
    eigvec_sc = eig_vecs[:,i].reshape(4,1)   
    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
    print('Eigenvalue {:}: {:.2e}'.format(i+1, eig_vals[i].real))
    
"""
Each of these eigenvectors is associated with an eigenvalue, which tells us about the “length” or “magnitude” of the 
eigenvectors.
If we would observe that all eigenvalues have a similar magnitude, then this may be a good indicator that our 
data is already projected on a “good” feature space.

And in the other scenario, if some of the eigenvalues are much much larger than others, we might be interested in keeping 
only those eigenvectors with the highest eigenvalues, since they contain more information about our data distribution. 
Vice versa, eigenvalues that are close to 0 are less informative and we might consider dropping those for constructing 
the new feature subspace.
If we are performing the LDA for dimensionality reduction, the eigenvectors are important since they will form the new axes
of our new feature subspace; the associated eigenvalues are of particular interest since they will tell us how informative 
the new axes are.
"""

Mean Vector class 1: [-1.0146  0.8423 -1.3049 -1.2551]

Mean Vector class 2: [ 0.1123 -0.6572  0.2851  0.1674]

Mean Vector class 3: [ 0.9023 -0.1851  1.0198  1.0877]

('within-class Scatter Matrix:\n', array([[ 57.1941,  38.3652,  16.9598,   9.0095],
       [ 38.3652,  91.2179,  10.685 ,  14.9475],
       [ 16.9598,  10.685 ,   8.8022,   4.6754],
       [  9.0095,  14.9475,   4.6754,  10.6746]]))
('between-class Scatter Matrix:\n', array([[ 371.2234, -219.0824,  455.2134,  454.734 ],
       [-219.0824,  235.1285, -295.0497, -273.7166],
       [ 455.2134, -295.0497,  564.7914,  558.9527],
       [ 454.734 , -273.7166,  558.9527,  557.3016]]))

Eigenvector 1: 
[[-0.1498]
 [-0.1482]
 [ 0.8511]
 [ 0.4808]]
Eigenvalue 1: 1.29e+02

Eigenvector 2: 
[[ 0.0095]
 [ 0.3272]
 [-0.5748]
 [ 0.75  ]]
Eigenvalue 2: 1.11e+00

Eigenvector 3: 
[[-0.7989]
 [ 0.0507]
 [ 0.0831]
 [ 0.5935]]
Eigenvalue 3: 3.74e-15

Eigenvector 4: 
[[ 0.3742]
 [ 0.0876]
 [ 0.5075]
 [-0.7712]]
Eigenvalue 4: 1.07e-14


'\nIf we are performing the LDA for dimensionality reduction, the eigenvectors are important since they will form the new axes \nof our new feature subspace; the associated eigenvalues are of particular interest since they will tell us how informative \nthe new axes are.\n'

In [10]:
# Checking the eigen vector - eigenvalue calculation

for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(4,1) 
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv), 
                                         eig_vals[i] * eigv, 
                                         decimal=6, err_msg='', verbose=True)
print('ok')

ok


In [11]:
# Step 4 selecting linear discriminants for the new feature space
# Sort the eigen vectors by decreasing eigenvalues
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
 
# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
 
# Visually confirm that the list is correctly sorted by decreasing eigenvalues
 
print('Eigenvalues in decreasing order:\n')
for i in eig_pairs:
    print(i[0])

Eigenvalues in decreasing order:

129.087831199
1.11026745536
1.06627406467e-14
3.73796164842e-15


In [12]:
print('Variance explained:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))

Variance explained:

eigenvalue 1: 99.15%
eigenvalue 2: 0.85%
eigenvalue 3: 0.00%
eigenvalue 4: 0.00%


In [13]:
# chosing k eigenvectors with the largest eigenvalues
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))
print('Matrix W:\n', W.real.T)

('Matrix W:\n', array([[-0.1498, -0.1482,  0.8511,  0.4808],
       [ 0.0095,  0.3272, -0.5748,  0.75  ]]))


In [14]:
# step 5 transforming the samples onto new subspace
X_lda = W.T.dot(X.T).T
assert X_lda.shape == (150,2), "The matrix is not 2x150 dimensional."
def plot_step_lda():
 
    ax = plt.subplot(111)
    for label,marker,color in zip(
        range(1,4),('^', 's', 'o'),('blue', 'red', 'green')):
 
        plt.scatter(x=X_lda[:,0][y == label],
                y=X_lda[:,1][y == label],
                marker=marker,
                color=color,
                alpha=0.5,
                label=label_dict[label]
                )
 
    plt.xlabel('LD1')
    plt.ylabel('LD2')
 
    leg = plt.legend(loc='upper right', fancybox=True)
    leg.get_frame().set_alpha(0.5)
    plt.title('LDA: Iris projection onto the first 2 linear discriminants')
 
    # hide axis ticks
    plt.tick_params(axis="both", which="both", bottom="off", top="off",  
            labelbottom="on", left="off", right="off", labelleft="on")
 
    # remove axis spines
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False) 
    ax.spines["bottom"].set_visible(False) 
    ax.spines["left"].set_visible(False)
 
    plt.grid()
    plt.tight_layout
    plt.show()
 
plot_step_lda()

In [18]:
# Compare it with PCA
from sklearn.decomposition import PCA as sklearnPCA
 
sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(X)
 
def plot_pca():
 
    ax = plt.subplot(111)
 
    for label,marker,color in zip(
        range(1,4),('^', 's', 'o'),('blue', 'red', 'green')):
 
        plt.scatter(x=X_pca[:,0][y == label],
                y=X_pca[:,1][y == label],
                marker=marker,
                color=color,
                alpha=0.5,
                label=label_dict[label]
                )
 
    plt.xlabel('PC1')
    plt.ylabel('PC2')
 
    leg = plt.legend(loc='upper right', fancybox=True)
    leg.get_frame().set_alpha(0.5)
    plt.title('PCA: Iris projection onto the first 2 principal components')
 
    # hide axis ticks
    plt.tick_params(axis="both", which="both", bottom="off", top="off",  
            labelbottom="on", left="off", right="off", labelleft="on")
 
    # remove axis spines
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False) 
    ax.spines["bottom"].set_visible(False) 
    ax.spines["left"].set_visible(False)
 
    plt.tight_layout
    plt.grid()
 
    plt.show()
    
plot_pca()

In [20]:
# LDA from schikit learn library
from sklearn.lda import LDA
 
# LDA
sklearn_lda = LDA(n_components=2)
X_lda_sklearn = sklearn_lda.fit_transform(X, y)
 
# PCA
sklearn_pca = sklearnPCA(n_components=2)
X_ldapca_sklearn = sklearn_pca.fit_transform(X_lda_sklearn)

def plot_scikit_lda(X, title, mirror=1):
 
    ax = plt.subplot(111)
    for label,marker,color in zip(
        range(1,4),('^', 's', 'o'),('blue', 'red', 'green')):
 
        plt.scatter(x=X[:,0][y == label]*mirror,
                y=X[:,1][y == label],
                marker=marker,
                color=color,
                alpha=0.5,
                label=label_dict[label]
                )
 
    plt.xlabel('LD1')
    plt.ylabel('LD2')
 
    leg = plt.legend(loc='upper right', fancybox=True)
    leg.get_frame().set_alpha(0.5)
    plt.title(title)
 
    # hide axis ticks
    plt.tick_params(axis="both", which="both", bottom="off", top="off",  
            labelbottom="on", left="off", right="off", labelleft="on")
 
    # remove axis spines
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False) 
    ax.spines["bottom"].set_visible(False) 
    ax.spines["left"].set_visible(False)
 
    plt.grid()
    plt.tight_layout
    plt.show()
    
plot_step_lda()
plot_scikit_lda(X_ldapca_sklearn, title='LDA+PCA via scikit-learn', mirror=(-1))
plot_scikit_lda(X_lda_sklearn, title='Default LDA via scikit-learn')

In [ ]:
# More can be learnt by looking at http://spartanideas.msu.edu/2014/08/03/linear-discriminant-analysis-bit-by-bit/